In [10]:
import os
import re   # regex

import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt

import torch
import tensorflow as tf

# To-do's

- [dialog](https://deview.kr/2019/schedule/285)
    - turn 제도


- [ ] 대화 촉발 키워드 or 패턴
- [ ] 이 사람이 할 법한 말! generator
    - input : keyword
- [ ] 년도별 이달의 우수회원

### 패턴

예시 문장 : `2018년 11월 7일 오전 1:30, A : 살다살다 내가 이런짓까지 한다.` -> `일자, 발언자 : 발언내용`

1. 일자
    - 년, 월, 일 구분
    - 한자리 일 때에도 07 등으로 자릿수 메우지 않음. 
    - 오전, 오후 구분 12시간제
2. 발언자
    - 데이터 추출자의 저장 상태를 따름
    - 앞뒤로 띄어쓰기 있음
3. 발언내용
    - : 이하 발언내용
    - 이모티콘은 이모티콘 분류 없이 `이모티콘` 으로 통일 but 유니코드 이모지는 원형 보존
    - 업로드한 사진은 `사진`, 공지 및 게시글은 `톡게시판`으로 표현


## 1. Load data

- 바로 위에서 확인한 패턴대로 대화가 이루어지는게 아니라 `\ufeff고삼즈 7 카카오톡 대화\n저장한 날짜 : 2022년 1월 27일 ` 이런 부분이 있음. 

> 어떻게 날릴까?

- [TODO] 0000년 00월 00일 오후 00:00 이 전까지의 데이터를 날려야 하는데 코드로 날릴 방법이 있을지 의문
- 일단 파일에서 삭제

> 아래와 같이 분리
의문: 굳이 정규식을 사용할 필요가 있나

1. 시각 (날짜, 시간 - 24시간제 변환)    
    - `\n` 기준으로 `Series` 생성
        - 하단의 ValueError로 read_csv 사용은 불가능. readline으로 한 줄씩 읽어가며 split하는 방식 채택
        - ValueError: Specified \n as separator or delimiter. This forces the python engine which does not accept a line terminator. Hence it is not allowed to use the line terminator as separator.
    - `,` 기준으로 열 분리 -> `DataFrame`
3. 발언자 분리
    - `:` 기준으로 앞
    - 정규식으로 분리하면 `, : ` 에서 `,` 까지 분리되는 문제
4. 발언 내용 분리
    -  `:` 기준으로 이후
    - 정규식 분리시 `:` 포함되는 문제


In [2]:
# CONSTATNS

KAKAOCHAT_PATH = '../data/KakaoTalkChats.txt'

# Message Parser -> 이게 필요한가...
WHEN_REGEX = "[0-9]{4}[년.] [0-9][0-9]?[월.] [0-9][0-9]?[일.] 오\S [0-9][0-9]?:[0-9][0-9]?,"
WHO_REGEX = ",.* :"
WHAT_REGEX = " : .*"

In [5]:
# R을 섞어쓰는구나 https://mustlearning.tistory.com/26
# library(dplyr)  # https://mustlearning.tistory.com/10


# 우선 Python 기준으로 진행
# https://wikidocs.net/144585

# raw data = rdata
with open(KAKAOCHAT_PATH, "r") as chats:
    rdata = chats.read()

# apply regex
def kakao_parser(KAKAOCHAT_PATH):
    chat_dict = {"when":[], "who":[], "what":[]}
    err_cnt = 0

    with open(KAKAOCHAT_PATH, "r") as chats:
        for chat in chats:
            try:
                when, other = chat.split(", ")[0], chat.split(", ")[1]
                who, what = other.split(" : ")[0], other.split(" : ")[1].strip("\n")
                
                if when and who and what:
                    chat_dict['when'].append(when)
                    chat_dict['who'].append(who)
                    chat_dict['what'].append(what)
            except:
                err_cnt += 1
    print(f"Process completed. {err_cnt} errors found.")
    return pd.DataFrame(chat_dict)

In [6]:
chat_df = kakao_parser(KAKAOCHAT_PATH)

Process completed. 3183 errors found.


In [8]:
chat_df.tail()

,when,who,what
213166,2022년 1월 27일 오전 12:00,홍,사진
213167,2022년 1월 27일 오전 12:13,윤가을,사진
213168,2022년 1월 27일 오전 12:13,윤가을,아 화질좋은분.?
213169,2022년 1월 27일 오전 12:13,윤가을,빨리 캡쳐좀여..
213170,2022년 1월 27일 오전 12:13,홍,사진


In [11]:
pr_report = chat_df.profile_report()
pr_report.to_file('./pr_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

/home/lky/anaconda3/envs/nlp_py39/lib/python3.9/site-packages/pandas_profiling/visualisation/utils.py:66: UserWarning: Glyph 54861 (\N{HANGUL SYLLABLE HONG}) missing from current font.
  plt.savefig(image_str, format=image_format)
/home/lky/anaconda3/envs/nlp_py39/lib/python3.9/site-packages/pandas_profiling/visualisation/utils.py:66: UserWarning: Glyph 50577 (\N{HANGUL SYLLABLE YANG}) missing from current font.
  plt.savefig(image_str, format=image_format)
/home/lky/anaconda3/envs/nlp_py39/lib/python3.9/site-packages/pandas_profiling/visualisation/utils.py:66: UserWarning: Glyph 48152 (\N{HANGUL SYLLABLE BAN}) missing from current font.
  plt.savefig(image_str, format=image_format)
/home/lky/anaconda3/envs/nlp_py39/lib/python3.9/site-packages/pandas_profiling/visualisation/utils.py:66: UserWarning: Glyph 44608 (\N{HANGUL SYLLABLE GIM}) missing from current font.
  plt.savefig(image_str, format=image_format)
/home/lky/anaconda3/envs/nlp_py39/lib/python3.9/site-packages/pandas_profiling

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]